# Inverse estimation

In many cases, one is interested in finding the maximum conductances from a given AP, a problem we refer to as the inverse problem. The package also provides a simple way to achieve this by using the function [`match_Vm`](/docs/match_Vm.html). For further details on the computations, please consult our full paper.

Like in the [simple forward problem](/demos/simple_forward.html), we start by loading a synthetic APs, but assume the maximum conductance to be unknown. Note that the maximum conductance of the samples are *not* included in the training set.

In [ ]:
import numpy as np

ap_data = np.load("data/syn/ap_data.npz")
t, aps_sim = ap_data["t"], ap_data["aps"]
target_ap_ind = 2
mask = t < 550
t, aps_sim = t[mask], aps_sim[:, mask]
target_ap = aps_sim[target_ap_ind]

In [ ]:
from cardiomyocyte_emulator import load_default_emulator_model
from cardiomyocyte_emulator.inverse import match_Vm

emulator = load_default_emulator_model(device="cpu")
weights = None #np.where((t > -2) & (t < 50), 10., 1.)
max_conds_est, aps_emu = match_Vm(emulator, t, target_ap, epochs=20, weights=weights)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(t, target_ap, color="r", linestyle="--", alpha=0.5, label="Simulated target")
plt.plot(t, aps_emu[0], color="k", alpha=0.5, label="Emulated found")
plt.xlim(-10, 550)
plt.legend()
plt.show()

The emulated AP can match the target AP well, but we further receive the maximum conductances that produce the emulated AP which we compare against the true maximum conductances.

In [ ]:
import pandas as pd
max_conds = pd.read_csv("data/syn/max_conds.csv")
ap_cond_df = max_conds.iloc[target_ap_ind].to_frame().transpose()
ap_cond_df = pd.concat([ap_cond_df, pd.DataFrame(max_conds_est, columns=ap_cond_df.columns)], axis=0)
ap_cond_df.index = ["Ground-truth", "Estimated"]
ap_cond_df

The mismatch in percent of the training range is given by:

In [ ]:
mismatch_df = emulator.normalize_max_conds_ranges_np(ap_cond_df.iloc[1]) - emulator.normalize_max_conds_ranges_np(ap_cond_df.iloc[0])
mismatch_df = mismatch_df.to_frame().transpose().abs()
mismatch_df.index = ["Relative mismatch"]
mismatch_df

## Parallel estimation

To better utilize computational resources, such as multi-core CPUs and GPUs, it is also possible to estimate multiple APs in parallel.

In [ ]:
max_conds_est, aps_emu = match_Vm(emulator, t, aps_sim, epochs=20, weights=weights)

In [ ]:
plt.figure()
sim_hs = plt.plot(t, aps_sim.T, color="r", linestyle="--", alpha=0.5)
emu_hs = plt.plot(t, aps_emu.T, color="k", alpha=0.5)
plt.xlim(-10, 550)
plt.legend([sim_hs[0], emu_hs[0]], ["Simulated", "Emulated"])
plt.show()

In [ ]:
max_conds_diff = emulator.normalize_max_conds_ranges_np(max_conds) - emulator.normalize_max_conds_ranges_np(pd.DataFrame(max_conds_est, columns=ap_cond_df.columns))
max_conds_err = max_conds_diff.abs().mean(0).to_frame().transpose()
max_conds_err.index = ["Relative mismatch"]
max_conds_err